# Memory Process Analysis
Lets look at the memory we just collected. To start you have to identify the memory image profile of the collected memory image. In this case we know what system we collected it on but in some cases you may just be procided with a raw image and may not know what system it was collected on. 

In [1]:
python ../volatility/vol.py -f C:\memoryraw imageinfo

Volatility Foundation Volatility Framework 2.6
INFO    : volatility.debug    : Determining profile based on KDBG search...
          Suggested Profile(s) : Win10x64_14393, Win10x64_10586, Win10x64_16299, Win2016x64_14393, Win10x64_15063 (Instantiated with Win10x64_15063)
                     AS Layer1 : SkipDuplicatesAMD64PagedMemory (Kernel AS)
                     AS Layer2 : FileAddressSpace (C:\memoryraw)
                      PAE type : No PAE
                           DTB : 0x1ab002L
                          KDBG : 0xf8037ef6d500L
          Number of Processors : 1
     Image Type (Service Pack) : 0
                KPCR for CPU 0 : 0xfffff8037efbf000L
             KUSER_SHARED_DATA : 0xfffff78000000000L
           Image date and time : 2018-03-20 19:03:32 UTC+0000
     Image local date and time : 2018-03-20 13:03:32 -0600


Now that we have identified the profile to use we can specificy that flag and begin analysis. The first analysis to focus on is the running processes. There are several ways to look at running processes. The first way we will look at running processes is with `pslist`. `pslist` analysis is similar to executing pslist from Sysinternals and it looks at the XXXX data structure in memory.

In [2]:
python ../volatility/vol.py -f C:\memoryraw --profile=Win10x64_14393 pslist

Volatility Foundation Volatility Framework 2.6
Offset(V)          Name                    PID   PPID   Thds     Hnds   Sess  Wow64 Start                          Exit                          
------------------ -------------------- ------ ------ ------ -------- ------ ------ ------------------------------ ------------------------------
0xffffc80311c60040 System                    4      0     97        0 ------      0 2018-02-10 18:55:29 UTC+0000                                 
0xffffc80312bfe040 smss.exe                492      4      2        0 ------      0 2018-02-10 18:55:29 UTC+0000                                 
0xffffc80312f3f780 csrss.exe               584    576     10        0      0      0 2018-02-10 18:55:30 UTC+0000                                 
0xffffc80313401380 smss.exe                648    492      0 --------      1      0 2018-02-10 18:55:30 UTC+0000                                 
0xffffc80313406080 wininit.exe             656    576      1        0      0 

It is sometimes easier to review processes as a tree structure instead of a flat list. The `pstree` command lists the process list as a tree structure with the trucks being the parent process and branches the spawned process. Execute the following command to view the process tree and review if all the critical processes are there and if anything looks supicous based on reviewing common processes

In [4]:
python ../volatility/vol.py -f C:\memoryraw --profile=Win10x64_14393 pstree

Volatility Foundation Volatility Framework 2.6
Name                                                  Pid   PPid   Thds   Hnds Time
-------------------------------------------------- ------ ------ ------ ------ ----
 0xffffc80313406080:wininit.exe                       656    576      1      0 2018-02-10 18:55:30 UTC+0000
. 0xffffc803134cb780:services.exe                     780    656      4      0 2018-02-10 18:55:30 UTC+0000
.. 0xffffc803132ec780:svchost.exe                    1544    780      6      0 2018-02-10 18:55:33 UTC+0000
.. 0xffffc80313144600:svchost.exe                    1900    780     12      0 2018-02-10 18:55:34 UTC+0000
.. 0xffffc8031355b780:svchost.exe                     652    780     76      0 2018-02-10 18:55:31 UTC+0000
... 0xffffc803129f1780:taskhostw.exe                 4252    652      4      0 2018-02-11 00:27:10 UTC+0000
... 0xffffc80313bdb780:taskhostw.exe                 2228    652     13      0 2018-02-10 18:56:15 UTC+0000
... 0xffffc803124eb780:sihost

The other process analysis is `psscan` which carves the EPROCESS structure from memory and does not rely on a linked data struture like `pslist`. Go ahead and execute the command to list what processes are in memory.

In [5]:
python ../volatility/vol.py -f C:\memoryraw --profile=Win10x64_14393 psscan

Volatility Foundation Volatility Framework 2.6
Offset(P)          Name                PID   PPID PDB                Time created                   Time exited                   
------------------ ---------------- ------ ------ ------------------ ------------------------------ ------------------------------
0x0000c80311c60040 System                4      0 0x00000000001ab002 2018-02-10 18:55:29 UTC+0000                                 
0x0000c80311e8d780 svchost.exe        1296    780 0x000000000c4c0002 2018-02-10 18:55:32 UTC+0000                                 
0x0000c803124eb780 sihost.exe         2024    652 0x00000000040a2002 2018-02-10 18:56:15 UTC+0000                                 
0x0000c803128b2780 cygwin-console     4916   1444 0x0000000009180002 2018-03-20 05:11:27 UTC+0000                                 
0x0000c803128ea780 firefox.exe         544    516 0x000000000c180002 2018-03-20 05:32:51 UTC+0000                                 
0x0000c80312950340 SearchIndexer.   

It can be troublesome to compare all the process enumeration capabiliteis in volatility so reviewing them all at the same time can be done with the command `psxview` 

In [6]:
python ../volatility/vol.py -f C:\memoryraw --profile=Win10x64_14393 psxview

Volatility Foundation Volatility Framework 2.6
Offset(P)          Name                    PID pslist psscan thrdproc pspcid csrss session deskthrd ExitTime
------------------ -------------------- ------ ------ ------ -------- ------ ----- ------- -------- --------
0x000000003b8be080 wininit.exe             656 True   True   True     True   True  True    False    
0x00000000678fc780 dasHost.exe             824 True   True   True     True   True  True    False    
0x000000002bd6d600 svchost.exe            1900 True   True   True     True   True  True    False    
0x0000000071cf2780 svchost.exe            1984 True   True   True     True   True  True    False    
0x000000001ab4e780 mintty.exe             1444 True   True   True     True   True  True    False    
0x000000007b4cf780 svchost.exe             860 True   True   True     True   True  True    False    
0x000000003ba6c080 svchost.exe            4984 True   True   True     True   True  True    False    
0x00000000351b0780 svchost.e